# Tutorial #5: Qualitative genome-scale modeling using public data, Flux Balance Analysis (FBA)

Author: Marc Weber  
Date: March 2016  
Affiliation: Center for Genomic Regulation (CRG), Barcelona, Spain.  
Notes: This tutorial was created for the course "Whole-cell modeling", taking place at CRG, Barcelona, April 3-8, 2016.

## Exercise 2: Metabolic network reconstruction and gap-filling

### Outline

+ Comparison of model predictions and experimental single gene knockout data
+ False positive predictions
+ False negative predictions and gaps
+ Dead-end metabolites and blocked reactions
+ Gap filling
  - Example 1, E. coli core metabolism and growMatch method
  - Example 2, predict gap-filling reaction for a dead-end metabolite in iJO1366 model
  - Example 3, model-driven discovery of isozymes for iJO1366 false-negative prediction
  - Example 4, reactions from Staphylococcus aureus as universal database for iJO1366 false-negative correction

### Comparison model predictions to experimental single gene knockout growth data

A metabolic model allows to predict the presence or absence of growth of single and multiple gene knockout mutants. By comparing the predictions to the experimental data, one may identify four different outcomes.

<img src="Images/Thiele2010_gene_essentiality_study.png" width="500" />
_Figure adapted from [Thiele2010]_

Because not every realistic constraint is represented in a typical metabolic model, it is quite possible for such a model to predict growth under conditions where growth does not really occur. The actual organism may not express a required gene for growth, or fluxes may be limited by kinetic or thermodynamic constraints, for example. This case is called a false positive prediction. On the other hand, false predictions of no growth can be taken as indications that the model is missing an essential reaction. This prediction is called a false negative.

In the gap filling analysis of Orth et al. (2012), the authors compared the growth predictions of the iJO1366 model of E. coli to a large experimental dataset of essentiality. In this exercise, we will follow the same approach and identify both false positive and false negatives comparisons to reveal potential errors in the model and in the experimental dataset. Then, we will use a gap-filling algorithm to extend and complete the metabolic network and predict new enzymatic reactions.

In [1]:
import escher
import escher.urls
import cobra
import cobra.test
import json
import shlex
from pprint import pprint
import os
import subprocess
import pandas as pd
import numpy as np
import itertools
import re
from IPython.display import HTML
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

# If you want to increase the default width of the notebook
#HTML("<style>.container { width:100% !important; }</style>")
# The default height of the metabolic maps as displayed in the notebook
escherDefaultMapHeight = 800

cobra/io/sbml3.py:23 UserWarning: Install lxml for faster SBML I/O
/home/mweber/.virtualenvs/python3/lib/python3.4/site-packages/matplotlib/__init__.py:872 UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.


First, we compute the growth rate of all single gene deletions of the metabolic model.

In [2]:
model = cobra.io.load_json_model('iJO1366.json')

growth_rates, statuses = cobra.flux_analysis.single_gene_deletion(model, model.genes[:])
list(growth_rates.items())[:30]

# We convert the lists to a pandas dataframe for easier data manipulation
model_knockout_df = pd.DataFrame.from_dict({"model_growth_rate": growth_rates})

# If the predicted growth rate is below the threshold, we consider the prediction as no growth.
growth_rate_threshold = 1e-5
model_knockout_df['model_growth'] = model_knockout_df['model_growth_rate'] > growth_rate_threshold

# Copy the gene cobra id index to a column
model_knockout_df['gene_cobra_id'] = model_knockout_df.index

print('Nb of essential genes: ',    len((model_knockout_df[~model_knockout_df['model_growth']])))
print('Nb of non-essential genes: ',len((model_knockout_df[model_knockout_df['model_growth']])))

model_knockout_df[:10]

Nb of essential genes:  208
Nb of non-essential genes:  1159


,model_growth_rate,model_growth,gene_cobra_id
b0002,9.823718e-01,True,b0002
b0003,0.000000e+00,False,b0003
b0004,-5.137062e-27,False,b0004
b0007,9.823718e-01,True,b0007
b0008,9.823718e-01,True,b0008
b0009,-2.027295e-29,False,b0009
b0019,9.823718e-01,True,b0019
b0025,2.629892e-14,False,b0025
b0026,9.823718e-01,True,b0026
b0029,-2.524355e-29,False,b0029


Rearrange the dataframe to include gene names.

In [3]:
# Define the function to apply at each row of the dataframe
def add_gene_name(row):
    geneModel = row.name
    geneName = model.genes.get_by_id(geneModel).name
    return geneName

# Apply the function to the rows (axis=1) of the dataframe and add the result as new columns
model_knockout_df = model_knockout_df.merge(pd.DataFrame(model_knockout_df.apply(add_gene_name, axis=1), columns=['gene']),
                                                        left_index=True, right_index=True)
# Drop genes that have no gene name ('None' string as gene name)
model_knockout_df = model_knockout_df[ model_knockout_df.gene != 'None' ]
model_knockout_df[:10]

,model_growth_rate,model_growth,gene_cobra_id,gene
b0002,9.823718e-01,True,b0002,thrA
b0003,0.000000e+00,False,b0003,thrB
b0004,-5.137062e-27,False,b0004,thrC
b0007,9.823718e-01,True,b0007,yaaJ
b0008,9.823718e-01,True,b0008,talB
b0009,-2.027295e-29,False,b0009,mog
b0019,9.823718e-01,True,b0019,nhaA
b0025,2.629892e-14,False,b0025,ribF
b0026,9.823718e-01,True,b0026,ileS
b0029,-2.524355e-29,False,b0029,ispH


Next, we download the single gene knockout growth data from the EcoCyc database for the LB Lennox medium.

In the EcoCyc website, select the Escherichia coli K-12 substr. MG1655 organism, go to Search->Search Growth Media, and click on "All growth media for this organism". The table lists all the growth media and the knockout data availability.
<img src="Images/Ecocyc_knockout_1.png" width="500" />
A large collection of experimental data on growth or non-growth on different media for single-gene knockout mutants has been included into the EcoCyc database, integrating six data sets describing 16,119 observations [Mackie2014]. We will extract the data for the LB Lennox medium. Click on the name of the medium. The list of single gene knockouts exhibit growth or non-growth are listed. For the non-growth case, transform the list into a SmartTable. Export the SmartTable to a spreadsheet file (important: choose "use common names format" in the export dialog). Rename the file to "EcoCyc_gene_knockout_gene_growth.xls".
<img src="Images/Ecocyc_knockout_3.png" width="500" />
Repeat the same for the non-growth list, rename file to "EcoCyc_gene_knockout_non-growth.xls".

Convert the two lists into dataframes and merge them.

In [4]:
# Importing gene names for growth data
exp_knockout_df = pd.read_table("EcoCyc_gene_knockout_growth.xls")
exp_knockout_df.columns = ['gene']
exp_knockout_df['exp_growth'] = True
print('Nb of knockouts that show growth: ', len(exp_knockout_df))

# Importing gene names for non-growth data
exp_knockout_no_growth_df = pd.read_table("EcoCyc_gene_knockout_non-growth.xls")
exp_knockout_no_growth_df.columns = ['gene']
exp_knockout_no_growth_df['exp_growth'] = False
print('Nb of knockouts that show no growth: ', len(exp_knockout_no_growth_df))

# Check that there is no overlap between the two sets
print(len(set(exp_knockout_df.gene).intersection(set(exp_knockout_no_growth_df.gene))))

# Concatenate the two datasets
# Hint: use the pd.Dataframe.append method
exp_knockout_df = exp_knockout_df.append(exp_knockout_no_growth_df)
print(exp_knockout_df[:5])
print(exp_knockout_df[-5:])

Nb of knockouts that show growth:  3840
Nb of knockouts that show no growth:  318
0
   gene exp_growth
0  aaeA       True
1  aaeB       True
2  aaeR       True
3  aaeX       True
4   aas       True
     gene exp_growth
313  ymfK      False
314  yqgD      False
315  yqgF      False
316  yrfF      False
317  zipA      False


Find the intersection between the list of model predictions genes and the list of experimental data genes.

In [5]:
knockout_df = pd.merge(model_knockout_df, exp_knockout_df, how='inner', on=['gene', 'gene'])
knockout_df.set_index('gene', drop=True, inplace=True)
knockout_df[:10]

,model_growth_rate,model_growth,gene_cobra_id,exp_growth
gene,,,,
thrA,9.823718e-01,True,b0002,True
thrB,0.000000e+00,False,b0003,True
thrC,-5.137062e-27,False,b0004,True
yaaJ,9.823718e-01,True,b0007,True
talB,9.823718e-01,True,b0008,True
mog,-2.027295e-29,False,b0009,True
nhaA,9.823718e-01,True,b0019,True
ribF,2.629892e-14,False,b0025,False
ileS,9.823718e-01,True,b0026,False


Compute the comparison between experimental growth and predicted growth.

In [6]:
def calculate_confusion(row):
    if ( row['model_growth'] and row['exp_growth'] ):
        confusion = 'TP'
    if ( row['model_growth'] and not row['exp_growth'] ):
        confusion = 'FP'
    if ( not row['model_growth'] and not row['exp_growth'] ):
        confusion = 'TN'
    if ( not row['model_growth'] and row['exp_growth'] ):
        confusion = 'FN'
    return pd.Series({'confusion':confusion})

knockout_df = knockout_df.merge(knockout_df.apply(calculate_confusion, axis=1), left_index=True, right_index=True)
knockout_df[:10]

,model_growth_rate,model_growth,gene_cobra_id,exp_growth,confusion
gene,,,,,
thrA,9.823718e-01,True,b0002,True,TP
thrB,0.000000e+00,False,b0003,True,FN
thrC,-5.137062e-27,False,b0004,True,FN
yaaJ,9.823718e-01,True,b0007,True,TP
talB,9.823718e-01,True,b0008,True,TP
mog,-2.027295e-29,False,b0009,True,FN
nhaA,9.823718e-01,True,b0019,True,TP
ribF,2.629892e-14,False,b0025,False,TN
ileS,9.823718e-01,True,b0026,False,FP


Count number of true positive and true negatives. Hint: use the `value_counts()` method of the dataframe column.

In [7]:
confusion_table = knockout_df['confusion'].value_counts()
confusion_table

TP    1068
FN     114
TN      92
FP      54
Name: confusion, dtype: int64

Calculate relative frequency (relative to the total number of comparisons).

In [8]:
confusion_table / confusion_table.sum()

TP    0.804217
FN    0.085843
TN    0.069277
FP    0.040663
Name: confusion, dtype: float64

### False positive predictions

There are several possible reasons for a false positive prediction. It is possible that the model may contain a reaction that is absent in vivo. This may happen for genes which function is not well characterized or are incorrectly annotated. False positive may also occur when a gene is down-regulated in certain conditions. While the model predicts growth, in the experiment the gene is repressed and the enzyme is not produced or at a very low level and cannot catalyze the reaction. Other reasons for false positive predictions include erroneous metabolite in the biomass, and kinetic or thermodynamics constraints that are note taken into account in the model. Many false positives occur when tRNA charging genes are knocked out in the model. Since the iJO1366 tRNA charging reactions are blocked by scope gaps, these important reactions cannot be used in the model. These types of false positive model predictions cannot be overcome through standard FBA using a metabolic model. A model including regulation or other additional constraints is required.

Some other false positive predictions likely occur because a gene was incorrectly identified as essential in one of the experimental screens. For example, in the analysis of [Orth2012], the ATP synthase genes atpCDGAHFEB were first classified as essential on minimal media, but inspection of the actual growth measurements from these experiments revealed that these knockout strains did actually grow, albeit slowly. With the recent updates of the EcoCyc database, these genes no longer appear as essential on LB and thus their knockouts result in true positives.

Another example of the limitations of the metabolic model is the false positive prediction of the gene spoT. This gene is required to synthesize and control the levels of the signaling molecule guanosine tetraphosphate (ppGpp). Since the metabolic model does not require signaling, this gene is found to be non-essential. Experimentally, spoT is essential on rich media, and this is likely due to its non-metabolic functions [Dalebroux2012].

Print out all the false positive predictions.

In [9]:
knockout_df[(knockout_df.confusion == 'FP')]

,model_growth_rate,model_growth,gene_cobra_id,exp_growth,confusion
gene,,,,,
ileS,0.982372,True,b0026,False,FP
folA,0.982372,True,b0048,False,FP
ftsI,0.982372,True,b0084,False,FP
can,0.982372,True,b0126,False,FP
pyrH,0.982372,True,b0171,False,FP
proS,0.982372,True,b0194,False,FP
adk,0.982372,True,b0474,False,FP
cysS,0.982372,True,b0526,False,FP
folD,0.970698,True,b0529,False,FP


Print out the false positive that have a gene name similar to a amino acyl tRNA synthetase (ending with S).

In [10]:
# Select rows of the dataframe based on a complex criterion: if false positive and
# gene index matches the pattern 'xxxS'
# Hint: use regex method re.match(pattern, string)
criterion1 = knockout_df.index.map( lambda gene: (re.match(r'\w\w\wS', gene) is not None) )
criterion2 = knockout_df['confusion'] == 'FP'
print('nb of genes: ', len(knockout_df[criterion1 & criterion2]))
knockout_df[criterion1 & criterion2]

nb of genes:  18


,model_growth_rate,model_growth,gene_cobra_id,exp_growth,confusion
gene,,,,,
ileS,0.982372,True,b0026,False,FP
proS,0.982372,True,b0194,False,FP
cysS,0.982372,True,b0526,False,FP
leuS,0.982372,True,b0642,False,FP
glnS,0.982372,True,b0680,False,FP
serS,0.982372,True,b0893,False,FP
asnS,0.982372,True,b0930,False,FP
tyrS,0.982372,True,b1637,False,FP
pheS,0.982372,True,b1714,False,FP


We observe that many tRNA synthetase knockouts give a false positive prediction, because of the scope gap of the model. All genes in the filtered list, except acpS, are tRNA synthetases, in total 17.

### False negative predictions and gap-filling

#### Gaps and dead-end metabolites

Even in very well-studied model organisms such as Escherichia coli there are still many genes with unknown functions. The result of this is that there are gaps in metabolic network reconstructions. These gaps take the form of dead-end metabolites, which have either no producing or no consuming reactions.

These dead-end metabolites can also be categorized into two groups, depending on the type of reactions that could connect them to the remaining network: knowledge gaps and scope gaps. The knowledge gaps represent the missing biochemical knowledge for the target organism. In contrast, the scope gaps include reactions and cellular processes, which are currently not accounted for in the metabolic reconstruction.

The two simplest approaches to identify gaps are the connectivity-based approach and the functionality-based approach. In the first one we identify metabolites which are only produced (root-non-consumed) or consumed (root-non-produced) by examining its corresponding row in the stochiometric matrix. A second approach is based on model functionality; in this approach the model capability to carry flux through every network reaction is tested. This approach identifies blocked reactions, which are directly or indirectly associated with one or more dead-end metabolites.

<img src="Images/Thiele2010_gap_analysis_2.png" width="700" />
_Figure adapted from [Thiele2010]_

Define the stochiometric matrix of the model with the method `model.to_array_based_model`. What is the shape of the matrix?

In [11]:
S = model.to_array_based_model().S
S.shape

(1805, 2583)

In [12]:
len(model.metabolites)

1805

In [13]:
len(model.reactions)

2583

In [14]:
# Convert the sparse matrix (scipy lil sparse matrix) into a dense matrix for easier computation
Sarray = S.toarray()[:,:]
Sarray.shape

(1805, 2583)

Find dead-end metabolites by inspecting the stochiometric matrix.

In [15]:
nrows = Sarray.shape[0]
deadendMetIndexList = []
for i in range(nrows):
    row = Sarray[i, :]
    # For each row, test if all the elements are equal to 1 or 0 (root-non-consumed) or
    # if all the elements are equal to -1 or 0 (root-non-produced).
    if np.all( (row == 1) | (row == 0) ) or np.all( (row == -1) | (row == 0) ):
        print(i, S[i,:],'\n')
        deadendMetIndexList.append(i)

8   (0, 340)	1.0
  (0, 644)	1.0
  (0, 1616)	1.0 

9   (0, 342)	1.0
  (0, 1617)	1.0 

10   (0, 1314)	1.0
  (0, 2077)	1.0 

29   (0, 975)	-1.0
  (0, 1588)	-1.0 

31   (0, 1026)	-1.0 

33   (0, 1011)	-1.0
  (0, 1012)	-1.0
  (0, 1013)	-1.0 

52   (0, 1038)	1.0
  (0, 1697)	1.0 

90   (0, 2079)	-1.0 

95   (0, 2456)	1.0 

96   (0, 2190)	1.0
  (0, 2486)	1.0 

99   (0, 1580)	-1.0
  (0, 1581)	-1.0 

100   (0, 1582)	1.0
  (0, 1583)	1.0 

133   (0, 976)	1.0
  (0, 1560)	1.0 

149   (0, 489)	-1.0
  (0, 1481)	-1.0 

157   (0, 906)	-1.0 

162   (0, 1515)	-1.0 

169   (0, 451)	1.0 

170   (0, 1498)	-1.0 

173   (0, 713)	1.0
  (0, 729)	1.0 

178   (0, 1045)	-1.0 

179   (0, 606)	-1.0
  (0, 2204)	-1.0 

183   (0, 605)	1.0
  (0, 606)	1.0 

202   (0, 2367)	1.0 

203   (0, 1850)	-1.0 

212   (0, 495)	1.0 

215   (0, 508)	1.0
  (0, 601)	1.0 

220   (0, 1334)	1.0 

222   (0, 1714)	1.0 

228   (0, 533)	-1.0
  (0, 534)	-1.0 

229   (0, 534)	1.0 

230   (0, 531)	-1.0
  (0, 537)	-1.0 

233   (0, 511)	1.0
  (0, 2

Count the number of dead-end metabolites.

In [16]:
len(deadendMetIndexList)

481

The second class of gaps are the blocked reactions that can be associated to dead-end metabolites, either upstream or downstream, or more generally to the existence of unconnected subsets of reactions (see [Ponce-deLeon2013]). These reactions are much more difficult to find and require an optimization approach. A reaction is said to be blocked when under a given condition it cannot reach a steady-state other than zero. Burgard et al. [Burgard2004] proposed a method to compute the set of blocked reactions by solving a set of linear optimizations. The approach consists in calculating the minimum and maximum flux value through each reaction of the system. When the maximum and minimum values found for a given reaction are both equal to zero, the reaction is said to be blocked under the defined medium condition. This is the approach implemented in the `find_block_reactions` cobra method.

Run the method on our model to find all the blocked reactions, print the results. How many blocked reactions are there?

In [17]:
blocked_reactions = cobra.flux_analysis.find_blocked_reactions(model)

In [18]:
print('Nb of blocked reactions: ',len(blocked_reactions))
blocked_reactions[:10]

Nb of blocked reactions:  878


['CRNt7pp',
 'EX_LalaDglu_e',
 'MNNH',
 'GPDDA1',
 'EX_no2_e',
 'DADNt2pp',
 'INOSTt4pp',
 'EX_acnam_e',
 'ACBIPGT',
 'PSCLYSt2pp']

### Gap filling

Many algorithms exist that can fill these gaps by suggesting and adding new hypothetical reactions. These _completion_ or _gap filling_ methods are able to identify the minimal set of additional reactions that allow growth in an extended model. Reactions are usually added from a database containing i) reactions from other organisms to provide functionality absent in the existing model, ii) reverse reactions for irreversible reactions in the existing model and iii) external transport mechanisms to allow for importation of metabolites in the existing model. Examples of databases containing a large collection of curated reactions include [KEGG](http://www.genome.jp/kegg/kegg1.html), [BiGG](http://bigg.ucsd.edu/) [King2016] or [MetaCyc](http://metacyc.org/) [Caspi2014].

The first such method to be published was called SMILEY [Reed2006]. This is a mixed-integer linear programming algorithm that identifies the minimum number of reactions that need to be added to a metabolic model from a universal database of reactions in order to allow a minimum defined growth rate to be achieved. The algorithms GapFind/GapFill [SatishKumar2007] and GrowMatch [Kumar2009] were later developed, and could predict missing reactions by connecting model gaps and by comparing model predictions to gene essentiality data, respectively. Other algorithms include Metaflux [Latendresse2012], part of the Pathway Tools software and its faster version FastGapFilling [Latendresse2014].

These predictions have the potential to improve the metabolic reconstruction and lead to new metabolic gene discoveries. However, gap filling algorithms only result in hypotheses that have to be evaluated for feasibility and eventually tested experimentally. There may be situations where manual inspection is needed. This is the case of the reconstruction of metabolic network from organisms that followed a reductive evolution of their genome, as for example bacterial pathogens and symbionts. During the coevolution with their eukaryotic host, these organisms lost many genes coding for enzymes involved in the synthesis of small metabolites, such as amino acids, nucleotides and lipids that were available in the host. These shared metabolic abilities take the form of interrupted pathways when the endosymbiotic network is reconstructed. Wether such gaps should be filled or not is thus a complex decision that requires the analysis of all experimental evidences supporting or refuting the existence of a reaction.

In the following, we will use gap-filling methods to  find new reactions to add to the metabolic model of E. coli to fill gaps from dead-end metabolites and to correct for false negative predictions.

### Gap-filling example  1: E. coli core metabolism and growMatch method

As a first example, we will take the simplified model of the core metabolism of E. coli, remove a few reactions and add them back to the model using the `growMatch` method.

In [19]:
model = cobra.test.create_test_model("textbook")
model.name = 'original_model'
biomass_objective = model.objective

In this model D-Fructose-6-phosphate (`f6p_c`) is an essential metabolite when growing on glucose. We will remove all the reactions using it, and at them to a separate model, the "Universal" model which contains the hypothetical reactions to be added to the initial model to fill the gaps.

Compute growth for the normal core model.

In [20]:
model.optimize().f

0.8739215069684305

Remove all the reactions associated with `f6p_c` and add them to the Universal model. Note: we do not want to remove the biomass reaction, which also contains fructose as a biomass precursor.

In [21]:
Universal = cobra.Model()
Universal.name = "Universal_Reactions"
# Loop over the id of reactions that contain fructose as metabolite
for reacid in [reac.id for reac in model.metabolites.f6p_c.reactions]:
    reaction = model.reactions.get_by_id(reacid)
    if (reaction.id != 'Biomass_Ecoli_core'):
        Universal.add_reaction(reaction.copy())
        reaction.remove_from_model()

Print which reactions were removed from the model and added to the Universal set.

In [22]:
for reac in Universal.reactions:
    print(reac.name,' ',reac.id)

glucose-6-phosphate isomerase   PGI
R Fructose transport via PEPPyr PTS-f6p - generating   FRUpts2
transketolase   TKT2
phosphofructokinase   PFK
fructose-bisphosphatase   FBP
transaldolase   TALA


Draw the metabolic map of the core metabolism with removed reactions highlighted in red.

In [23]:
metabolicMap = escher.Builder(map_name='e_coli_core.Core metabolism',
                              model=model,
                              # in the map, highlight all reactions that are missing from the model
                              highlight_missing=True,
                              # only show the primary metabolites
                              hide_secondary_metabolites=True)
metabolicMap.display_in_notebook()

Because of these gaps, the model will not grow and the FBA optimization is infeasible.

In [24]:
model.optimize()

<Solution 'infeasible' at 0x7f2838a16c88>

We will use GrowMatch to add back the minimal number of reactions from this set of "universal" reactions (in this case just the ones we removed) to allow it to grow. We can obtain multiple possible reaction sets by having the algorithm go through multiple iterations. Note that the algorithm first transform all reversible reactions in the universal set to two irreversible reactions ("reaction1" and "reaction1_reverse"). Then, it can propose to add only the reverse reaction or both.

Run `growMatch` with a few iterations.

In [25]:
addedReactions = cobra.flux_analysis.growMatch(model, Universal, iterations=2)
addedReactions

[[<Reaction PGI at 0x7f2838a144a8>,
  <Reaction PFK at 0x7f2838a143c8>,
  <Reaction TKT2_reverse at 0x7f2838a142e8>],
 [<Reaction TKT2 at 0x7f2838a14390>,
  <Reaction TALA at 0x7f2838a14358>,
  <Reaction PGI_reverse at 0x7f2838a14320>]]

The SMILEY algorithm, the core of the growMatch method, adds dummy metabolites and reactions for each reaction in the univeral set in order to perform the linear optimization and find the active added reactions (those added reactions that carry a flux), see for more information the details of the class `SUXModelMILP` in the gapfilling.py file. If we want to use these reactions to add them back into the original model, we first have to remove those dummy metabolites.

In [26]:
def remove_dummy_met(addedReactions):
    addedReactionsCopy = addedReactions
    for addedReactionSet in addedReactions:
        for reac in addedReactionSet:
            print(reac.id)
            print(reac.reaction)
            for met in reac.metabolites:
                if re.match(r'dummy_met_', met.id):
                    #  To remove the metabolite from the reaction, use the `pop` method on the reaction object
                    reac.pop(met)
            print(reac.reaction)
            print()
    return addedReactionsCopy
            
remove_dummy_met(addedReactions)

PGI
g6p_c --> dummy_met_PGI + f6p_c
g6p_c --> f6p_c

PFK
atp_c + f6p_c --> adp_c + dummy_met_PFK + fdp_c + h_c
atp_c + f6p_c --> adp_c + fdp_c + h_c

TKT2_reverse
f6p_c + g3p_c --> dummy_met_TKT2_reverse + e4p_c + xu5p__D_c
f6p_c + g3p_c --> e4p_c + xu5p__D_c

TKT2
e4p_c + xu5p__D_c --> dummy_met_TKT2 + f6p_c + g3p_c
e4p_c + xu5p__D_c --> f6p_c + g3p_c

TALA
g3p_c + s7p_c --> dummy_met_TALA + e4p_c + f6p_c
g3p_c + s7p_c --> e4p_c + f6p_c

PGI_reverse
f6p_c --> dummy_met_PGI_reverse + g6p_c
f6p_c --> g6p_c



[[<Reaction PGI at 0x7f2838a144a8>,
  <Reaction PFK at 0x7f2838a143c8>,
  <Reaction TKT2_reverse at 0x7f2838a142e8>],
 [<Reaction TKT2 at 0x7f2838a14390>,
  <Reaction TALA at 0x7f2838a14358>,
  <Reaction PGI_reverse at 0x7f2838a14320>]]

Add proposed reactions back to the model using the `add_reactions` method, draw the metabolic map and compute growth again.

In [27]:
# First create a copy of the original model
model_extended = model.copy()
model_extended.name = "model_extended"
# Add reactions
model_extended.add_reactions(addedReactions[0])

In [28]:
metabolicMap = escher.Builder(map_name='e_coli_core.Core metabolism',
                              model=model_extended,
                              # in the map, highlight all reactions that are missing from the model
                              highlight_missing=True,
                              # only show the primary metabolites
                              hide_secondary_metabolites=True)
metabolicMap.display_in_notebook()

The growth in the extended model should be restored.

In [29]:
model_extended.optimize()

<Solution 0.86 at 0x7f2838a169e8>

Try to add another set of reactions that can restore growth in the model.

In [30]:
# First create a copy of the original model
model_extended = model.copy()
model_extended.name = "model_extended"
# Add reactions
model_extended.add_reactions(addedReactions[1])

In [31]:
metabolicMap = escher.Builder(map_name='e_coli_core.Core metabolism',
                              model=model_extended,
                              # in the map, highlight all reactions that are missing from the model
                              highlight_missing=True,
                              # only show the primary metabolites
                              hide_secondary_metabolites=True)
metabolicMap.display_in_notebook()

The alternative set of added reactions leads to a different pathway to restore glycolysis. The growth rate can be different from the first set of added reactions.

In [32]:
model_extended.optimize()

<Solution 0.70 at 0x7f2838a16ba8>

The gap filling algorithm performs an optimization to find the minimal set of reactions that lead to positive growth. The solution depends therefore on the medium condition of the model. Repeat the gap filling analysis when the medium contains fructose instead of glucose.

Set the uptake rate of glucose to 0 and the uptake rate of fructose to -10.

In [33]:
model.reactions.EX_glc__D_e.lower_bound = 0
model.reactions.EX_fru_e.lower_bound = -10

Run the growMatch method on the original model.

In [34]:
addedReactions = cobra.flux_analysis.growMatch(model, Universal, iterations=2)
remove_dummy_met(addedReactions)
addedReactions

FRUpts2
fru_e + pep_c --> dummy_met_FRUpts2 + f6p_c + pyr_c
fru_e + pep_c --> f6p_c + pyr_c

PFK
atp_c + f6p_c --> adp_c + dummy_met_PFK + fdp_c + h_c
atp_c + f6p_c --> adp_c + fdp_c + h_c

PGI_reverse
f6p_c --> dummy_met_PGI_reverse + g6p_c
f6p_c --> g6p_c

TKT2_reverse
f6p_c + g3p_c --> dummy_met_TKT2_reverse + e4p_c + xu5p__D_c
f6p_c + g3p_c --> e4p_c + xu5p__D_c

FRUpts2
fru_e + pep_c --> f6p_c + pyr_c
fru_e + pep_c --> f6p_c + pyr_c

TKT2
e4p_c + xu5p__D_c --> dummy_met_TKT2 + f6p_c + g3p_c
e4p_c + xu5p__D_c --> f6p_c + g3p_c

TALA
g3p_c + s7p_c --> dummy_met_TALA + e4p_c + f6p_c
g3p_c + s7p_c --> e4p_c + f6p_c

PGI_reverse
f6p_c --> g6p_c
f6p_c --> g6p_c



[[<Reaction FRUpts2 at 0x7f2838a622b0>,
  <Reaction PFK at 0x7f2838a62320>,
  <Reaction PGI_reverse at 0x7f2838a62208>,
  <Reaction TKT2_reverse at 0x7f2838a621d0>],
 [<Reaction FRUpts2 at 0x7f2838a622b0>,
  <Reaction TKT2 at 0x7f2838a62518>,
  <Reaction TALA at 0x7f2838a62240>,
  <Reaction PGI_reverse at 0x7f2838a62208>]]

Notice that now the exchange reaction for fructose uptake is present in all the different sets of added reactions because the metabolic network needs to import fructose which is the only carbon source.

Add proposed reactions back to the model using the `add_reactions` method, draw the metabolic map and compute growth again.

In [35]:
# First create a copy of the original model
model_extended = model.copy()
model_extended.name = "model_extended"
# Add reactions
model_extended.add_reactions(addedReactions[0])

In [36]:
metabolicMap = escher.Builder(map_name='e_coli_core.Core metabolism',
                              model=model_extended,
                              # in the map, highlight all reactions that are missing from the model
                              highlight_missing=True,
                              # only show the primary metabolites
                              hide_secondary_metabolites=True)
metabolicMap.display_in_notebook()

The growth should be restored.

In [37]:
model_extended.optimize()

<Solution 0.86 at 0x7f2838a62e48>

### Gap-filling example 2: predict gap-filling reaction for a dead-end metabolite

In the following examples, we will follow the analysis of Orth and et [Orth2012], identify gaps in the iJO1366 metabolic model that are associated with false negative predictions and use a gap-filling algorithm to find hypothetical new enzymatic reactions and genes.

Gap-filling algorithms can be used to solve gaps that are consequences of dead-end metabolites. In this example, we will choose one root-non-produced metabolite, 2,5-diketo-D-gluconate, and use the SMILEY algorithm to find a reaction that fills the gap. We will use the set of all reverse reactions from the original model as the universal reactions database.

First, load the iJO1366 model.

In [38]:
model = cobra.io.load_json_model('iJO1366.json')
print('Nb of reactions: ',len(model.reactions))
print('Nb of metabolites: ',len(model.metabolites))
print('Nb of genes: ',len(model.genes))
model.optimize()

Nb of reactions:  2583
Nb of metabolites:  1805
Nb of genes:  1367


<Solution 0.98 at 0x7f283b1e0a58>

Print all the reactions for the metabolite 2,5-diketo-D-gluconate (`25dkglcn_c`).

In [39]:
for reac in model.metabolites.get_by_id('25dkglcn_c').reactions:
    print(reac.reaction)

25dkglcn_c + h_c + nadh_c --> 5dglcn_c + nad_c
25dkglcn_c + h_c + nadph_c --> 5dglcn_c + nadp_c
25dkglcn_c + h_c + nadph_c --> 2dhguln_c + nadp_c


The metabolite is a root-non-produced (RNP) metabolite since it is only involved in consuming reactions.

In order to fill this gap, we will try to add some reverse reactions from the irreversible reactions in the model. We build the set of all reverse reactions as follows:
+ iterate over all the reactions in the model
+ if reaction is *irreversible*, is not an exchange reaction and is not the biomass reaction:
  - define the reverse reaction
  - add note to both the forward and reverse reactions to keep track of their relation
  - define gene-reaction association
  - add reverse reaction to the list

In [40]:
# Building the list of reverse reactions for all irreversible reactions in the model
reverse_reaction_list = []
for reaction in model.reactions:
    if not reaction.reversibility and reaction.id[0:2] != 'EX' and reaction.id[0:7] != 'BIOMASS':
        reverse_reaction = cobra.Reaction(reaction.id + "_reverse")
        reverse_reaction.lower_bound = 0
        reverse_reaction.upper_bound = 1000
        reverse_reaction.objective_coefficient = 0
        # Make the directions aware of each other
        reverse_reaction.notes["reflection"] = reaction.id
        reaction_dict = {k: v * -1 \
                         for k, v in reaction._metabolites.items()}
        reverse_reaction.add_metabolites(reaction_dict)
        reverse_reaction._model = reaction._model
        reverse_reaction._genes = reaction._genes
        for gene in reaction._genes:
            gene._reaction.add(reverse_reaction)
        reverse_reaction.subsystem = reaction.subsystem
        reverse_reaction._gene_reaction_rule = reaction._gene_reaction_rule
        reverse_reaction_list.append(reverse_reaction)
        
reverse_reaction_list

[<Reaction DM_4crsol_c_reverse at 0x7f283b6403c8>,
 <Reaction DM_5drib_c_reverse at 0x7f283b640518>,
 <Reaction DM_aacald_c_reverse at 0x7f283b168f28>,
 <Reaction DM_amob_c_reverse at 0x7f283b168278>,
 <Reaction DM_mththf_c_reverse at 0x7f283b1689b0>,
 <Reaction DM_oxam_c_reverse at 0x7f283b1689e8>,
 <Reaction 12DGR120tipp_reverse at 0x7f283b1684a8>,
 <Reaction 12DGR140tipp_reverse at 0x7f283b168940>,
 <Reaction 12DGR141tipp_reverse at 0x7f283b168860>,
 <Reaction 12DGR160tipp_reverse at 0x7f283b168a20>,
 <Reaction 12DGR161tipp_reverse at 0x7f283b168e80>,
 <Reaction 12DGR180tipp_reverse at 0x7f283b168b70>,
 <Reaction 12DGR181tipp_reverse at 0x7f283b168dd8>,
 <Reaction 14GLUCANabcpp_reverse at 0x7f283b1683c8>,
 <Reaction 14GLUCANtexi_reverse at 0x7f283b168e10>,
 <Reaction 23DAPPAt2pp_reverse at 0x7f283b168d68>,
 <Reaction 23PDE2pp_reverse at 0x7f283b168eb8>,
 <Reaction 23PDE4pp_reverse at 0x7f283b168e48>,
 <Reaction 23PDE7pp_reverse at 0x7f283b168f60>,
 <Reaction 23PDE9pp_reverse at 0x7f

Use the list of reverse reactions to build the universal model.

In [41]:
model_universal = cobra.Model("Universal")
model_universal.add_reactions(reverse_reaction_list)

In order to predict gap-filling reactions, we directly use the gap-filling optimization method from the class `SUXModelMILP`, the core of the SMILEY algorithm. This class solves the MILP problem to find the minimal subset of reactions from the universal set of reactions to add to the original model such that fluxes through the objective reactions are above a threshold. The resulting model with extended universal and exchange reactions (SUX) presents a feasible optimal solution for the defined constraints.

The method first assigns to the objective reactions of the original model a lower bound proportional to the threshold to enforce a minimal flux ($\text{v}>\alpha$). Then, for all reactions in the original model, the objective coefficients are set to zero. The universal reactions are assigned new objective coefficients related to the penalty for adding the reaction to the network (usually exchange reactions have a much larger penalty than universal or demand reactions). The new objective in the gap-filling optimization is then to add a minimum set of universal reactions (minimize the penalties) while satisfying the minimal flux constraints from the original model.

In order to solve the gap for a specific root-non-produced metabolite, we can add a *demand* (consume) reaction for this metabolite, add the reaction to the *objective* of the original model, and run the gap-filling optimization. The gap-filling method will then try to add universal reactions to the original model in order to allow a minimal production flux for this metabolite.

<img src="Images/gap_filling_scheme.png" width="800px">

Create a demand reaction for the metabolite '25dkglcn_c' and set its objective coefficient to 1.

In [42]:
metabolite_gap = model.metabolites.get_by_id('25dkglcn_c')
demand_name = "SMILEY_DM_" + metabolite_gap.id
demand_name

'SMILEY_DM_25dkglcn_c'

In [43]:
demand_reaction = cobra.Reaction()
if demand_name not in model.reactions:
    demand_reaction = cobra.Reaction(demand_name)
    # Add the metabolite to the reaction
    demand_reaction.add_metabolites(
        {metabolite_gap: -1})
    model.add_reaction(demand_reaction)
else:
    demand_reaction = model.reactions.get_by_id(demand_name)

# Set the objective coefficient of the demand reaction
demand_reaction.objective_coefficient = 1
demand_reaction.reaction

'25dkglcn_c --> '

Define the SUX extended model from the gap-filling optimization class `SUXModelMILP`. We set to False the options of adding demand reactions for all metabolites (`dm_rxns`) and adding exchange reactions for all metabolites (`ex_rxns`).

In [44]:
SUX = cobra.flux_analysis.gapfilling.SUXModelMILP(model, model_universal, dm_rxns=False, ex_rxns=False)

At this step, the demand reaction (`SMILEY_DM_25dkglcn_c`) has been assigned a minimal lower bound equal to the threshold because it was an objective reaction. Print the lower bound of the demand reaction in the SUX model.

In [45]:
SUX.reactions.get_by_id('SMILEY_DM_25dkglcn_c').lower_bound

0.05

Solve the gap-filling optimization by calling the `solve` method of the SUX model.

In [46]:
addedReactions = SUX.solve(debug=True)
addedReactions

b'Constructing initial basis...\n'
b'Size of triangular part is 3199\n'
Iteration 0: Status is optimal
     DKGLCNR2x_reverse 0


[[<Reaction DKGLCNR2x_reverse at 0x7f2833db4128>]]

Print all the added reactions sets.

In [47]:
for i, addedReactionSet in enumerate(addedReactions):
    print('Added reaction set #', i,)
    for reac in addedReactionSet:
        print(reac.id)
        print(reac.reaction)
        print()

Added reaction set # 0
DKGLCNR2x_reverse
5dglcn_c + nad_c --> 25dkglcn_c + dummy_met_DKGLCNR2x_reverse + h_c + nadh_c



As discussed in [Orth2012], experimental evidences exist supporting the reversibility of two model reactions, `DKGLCNR1` (2,5-diketo-D-gluconate reductase) [Habrych2002] and `DKGLCNR2y` (2,5-diketo-D-gluconate reductase (NADPH)) [Yum1999].

### Gap-filling example 3: model-driven discovery of isozymes for iJO1366 false-negative prediction

In the case of a false negative prediction, gap-filling methods can be used to suggest new hypothetical reactions missing in the model. The experimental evidence of growth in a strain with a specific gene knocked out when the metabolic model predicts no growth suggests that the model is lacking some reactions that can compensate for the loss of the gene and its associated enzymes (e.g. alternative pathway). Gap-filling methods such as growMatch [Kumar2009] can suggest a minimal set of reactions to be added to the model to restore growth in the knockout mutant.

However, in general gap-filling predictions for new genes is difficult because the presence of inconsistencies in databases propagate to the reconstructed models, making it hard to assess the feasibility of hypothetical new reactions. Recent efforts have focused on combining metabolic model predictions and single and double knockout mutant screens to discover new isozyme functions for known genes. In this exercise we will follow the analysis of Guzman et al. [Guzman2015] and study AspC isozymes with the iJO1366 model.

The Asp aminotransferase in E. coli, encoded by the gene aspC, has been characterized as a broad-substrate, multifunctional enzyme that catalyzes the formation of Asp, Phe, and Tyr. Experimental studies reported that _aspC_ knockout strain are viable in both rich media and glucose minimal medium. However, the iJO1366 predicts no growth when simulating deletion of the _aspC_ gene. This false negative prediction suggests the existence of unknown isozymes that could perform the same function as AspC and compensate for its deletion. To identify potential isozymes based on sequence homology, run BLASTp for AspC protein, limiting the search to the organism E. coli.

Identify one candidate protein and its coding sequence with an expect value of `< 1E-40` and high-sequence identity percentage. Paste the result:

```
MULTISPECIES: aromatic amino acid aminotransferase [Proteobacteria]
Sequence ID: ref|WP_000486985.1|Length: 397Number of Matches: 1
See 8 more title(s)
tyrosine aminotransferase, tyrosine-repressible, PLP-dependent [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: ref|NP_418478.1| tyrosine aminotransferase [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: gb|AAC43148.1| tyrosine aminotransferase, tyrosine-repressible, PLP-dependent [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: gb|AAC77024.1| tyrosine aminotransferase, tyrosine-repressible, PLP-dependent [synthetic Escherichia coli C321.deltaA]
Sequence ID: gb|AGX35965.1| aromatic amino acid aminotransferase [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: gb|AIZ93177.1| aromatic amino acid aminotransferase [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: gb|ALI38288.1| aromatic amino acid aminotransferase [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: gb|AMC96912.1| aromatic amino acid aminotransferase [Escherichia coli str. K-12 substr. MG1655]
Sequence ID: gb|AMK99484.1|
Related Information
Gene-associated gene details
Structure-3D structure displays
Identical Proteins-Identical proteins to WP_000486985.1

Range 1: 1 to 397GenPeptGraphicsNext MatchPrevious Match
Alignment statistics for match #1
Score	Expect	Method	Identities	Positives	Gaps
338 bits(867)	3e-113	Compositional matrix adjust.	169/397(43%)	242/397(60%)	1/397(0%)
Query  1    MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVLTSVKKAEQYL-LEN  59
            MF+ + A   DPIL L + F+ D R  K+NL IG+Y +E G  P L +V +AE  L  + 
Sbjct  1    MFQKVDAYAGDPILTLMERFKEDPRSDKVNLSIGLYYNEDGIIPQLQAVAEAEARLNAQP  60

Query  60   ETTKNYLGIDGIPEFGRCTQELLFGKGSALINDKRARTAQTPGGTGALRVAADFLAKNTS  119
                 YL ++G+  +      LLFG    ++  +R  T QT GG+GAL+V ADFL +   
Sbjct  61   HGASLYLPMEGLNCYRHAIAPLLFGADHPVLKQQRVATIQTLGGSGALKVGADFLKRYFP  120

Query  120  VKRVWVSNPSWPNHKSVFNSAGLEVREYAYYDAENHTLDFDALINSLNEAQAGDVVLFHG  179
               VWVS+P+W NH ++F  AG EV  Y +YD   + + F+ L+ +L    A  +VL H 
Sbjct  121  ESGVWVSDPTWENHVAIFAGAGFEVSTYPWYDEATNGVRFNDLLATLKTLPARSIVLLHP  180

Query  180  CCHNPTGIDPTLEQWQTLAQLSVEKGWLPLFDFAYQGFARGLEEDAEGLRAFAAMHKELI  239
            CCHNPTG D T +QW  + ++   +  +P  D AYQGF  G+EEDA  +RA A+     +
Sbjct  181  CCHNPTGADLTNDQWDAVIEILKARELIPFLDIAYQGFGAGMEEDAYAIRAIASAGLPAL  240

Query  240  VASSYSKNFGLYNERVGACTLVAADSETVDRAFSQMKAAIRANYSNPPAHGASVVATILS  299
            V++S+SK F LY ERVG  +++  D+E   R   Q+KA +R NYS+PP  GA VVA +L+
Sbjct  241  VSNSFSKIFSLYGERVGGLSVMCEDAEAAGRVLGQLKATVRRNYSSPPNFGAQVVAAVLN  300

Query  300  NDALRAIWEQELTDMRQRIQRMRQLFVNTLQEKGANRDFSFIIKQNGMFSFSGLTKEQVL  359
            ++AL+A W  E+ +MR RI  MRQ  V  L  +   R+F +++ Q GMFS++GL+  QV 
Sbjct  301  DEALKASWLAEVEEMRTRILAMRQELVKVLSTEMPERNFDYLLNQRGMFSYTGLSAAQVD  360

Query  360  RLREEFGVYAVASGRVNVAGMTPDNMAPLCEAIVAVL  396
            RLREEFGVY +ASGR+ VAG+   N+  + +A  AV+
Sbjct  361  RLREEFGVYLIASGRMCVAGLNTANVQRVAKAFAAVM  397
```

The _tyrB_ gene is an isozyme candidate for _aspC_. Guzman et al. showed that the double knockout mutant $\Delta$_aspC_$\Delta$_tyrB_ is a synthetic lethal pair and exhibit non-growth on glucose minimal medium. Moreover _tyrB_ gets overexpressed in $\Delta$_aspC_ mutant strain.

<img src="Images/Guzman2015_aspC_tyrB_double_KO_1.png" width="450px" >

Similarly, they identified another isozyme, _ilvE_, and showed that these three aminotransferases have overlapping in vivo functionality. Therefore, the gene-protein-reaction association should be changed accordingly (see panel A),

<img src="Images/Guzman2015_new_gene_protein_association_1.png" width="600px" >

Check that the original iJO1366 model predicts no growth for _aspC_ gene deletion. Reminder: use the `cobra.flux_analysis.single_gene_deletion` method.

In [68]:
model = cobra.io.load_json_model('iJO1366.json')

In [69]:
cobra.flux_analysis.single_gene_deletion(model, [model.genes.b0928])

({'b0928': -7.942270348596099e-14}, {'b0928': 'optimal'})

Check that the original iJO1366 model predicts growth for _TyrB_ gene deletion.

In [70]:
cobra.flux_analysis.single_gene_deletion(model, [model.genes.b4054])

({'b4054': 0.9823718127269793}, {'b4054': 'optimal'})

Print the gene reaction rules for genes AspC and TyrB.

In [71]:
print('aspC (b0928) gene-reaction association rules:')
for reac in model.genes.b0928.reactions:
    print(reac.id,',', reac.name,',', reac.gene_name_reaction_rule)

print()
print('tyrB (b4054) gene-reaction association rules:')
for reac in model.genes.b4054.reactions:
    print(reac.id,',', reac.name,',', reac.gene_name_reaction_rule)

aspC (b0928) gene-reaction association rules:
TYRTA , Tyrosine transaminase , tyrB or aspC
PHETA1 , Phenylalanine transaminase , tyrB or aspC or ilvE
ASPTA , Aspartate transaminase , aspC

tyrB (b4054) gene-reaction association rules:
TYRTA , Tyrosine transaminase , tyrB or aspC
PHETA1 , Phenylalanine transaminase , tyrB or aspC or ilvE
LEUTAi , Leucine transaminase (irreversible) , ilvE or tyrB


Change the gene-reaction association in the iJO1366 model to correct for the new AspC isozymes functions: change the rules for the `ASPTA` reaction.

In [72]:
model.reactions.ASPTA._gene_reaction_rule = 'b0928 or b4054'

Check that the model now predicts growth in single knockout mutant $\Delta$_aspC_ and non-growth in double knockout mutant $\Delta$_aspC_$\Delta$_tyrB_. Hint: for the double knockout, use the method `cobra.flux_analysis.double_gene_deletion`.

In [77]:
cobra.flux_analysis.single_gene_deletion(model, [model.genes.b0928])

({'b0928': 0.9823718127269793}, {'b0928': 'optimal'})

In [78]:
cobra.flux_analysis.double_gene_deletion(model, [model.genes.b0928], [model.genes.b4054])

{'data': array([[ 0.]]), 'x': ['b0928'], 'y': ['b4054']}

### Gap-filling example 4: reactions from Staphylococcus aureus as universal database

In most gap-filling analysis, a large universal database of reactions from all possible organisms (or from bacterial species only) is used in order to extend the scope of hypothetical reactions. The use of such databases, such as KEGG, brings the additional difficulty of mapping metabolites and reactions between the database and the metabolic model. The BiGG database [King2016] was built around a set of high-quality genome-scale models and is fully compatible with metabolic models in the COBRA format. It provides a standard for genome-scale metabolic network reconstructions and solves the problem of unique identifiers for metabolites and reactions between different models.

The universal set of reactions of the BiGG database is a set of non-redundant reactions. Unfortunately, it is unavailable for direct download. In the following example, we will use as a proxy of the more comprehensive universal database the metabolic model of Staphylococcus aureus. By using the model of a different bacterial species, we expect that many reactions in the S. aureus model are absent in E. coli and may be potential new reactions to add to the iJO1366 model. We will perform a gap-filling analysis of the iJO1366 model with the aim of correcting for false-negative predictions. We will look at a few examples of suggested hypothetical reactions and we will evaluate the feasibility of the additional reactions by comparing the number of false positive and false negatives predictions between the original and the extended model.

In [79]:
model = cobra.io.load_json_model('iJO1366.json')

Download the iSB619 metabolic model of Staphylococcus aureus subsp. aureus N315 from the BiGG database. See the instructions for the [BiGG web API](http://bigg.ucsd.edu/web_api) or download the model in JSON format from the website.

In [ ]:
# Staphylococcus aureus subsp. aureus N315
cmd = 'wget -O ./BiGG_models/BiGG_model_iSB619.json \"http://bigg.ucsd.edu/api/v2/models/iSB619/download\"'
output = subprocess.check_output( cmd, shell = True )
print(output)

Import the SB619 model and list the number of genes, metabolites and reactions.

In [83]:
# Staphylococcus aureus subsp. aureus N315
model2 = cobra.io.load_json_model('BiGG_models/BiGG_model_iSB619.json')
print('Nb of reactions: ',len(model2.reactions))
print('Nb of metabolites: ',len(model2.metabolites))
print('Nb of genes: ',len(model2.genes))

Nb of reactions:  743
Nb of metabolites:  655
Nb of genes:  619


Add all the reactions from the S. aureus model that are not in the original model to the universal database. We have to check that the unique BiGG identifiers of the two reactions are different. BiGG identifiers are recorded in the notes attribute of the reaction objects.

In [84]:
# First, create a list with the BiGG_ids of all reactions in the original model
model_reaction_BiGG_id_list = [reac.notes['original_bigg_ids'][0] for reac in model.reactions]

model2_new_reaction_list = []
for reac2 in model2.reactions:
    if reac2.notes['original_bigg_ids'][0] not in model_reaction_BiGG_id_list:
        # The BiGG_id is unique in all BiGG models.
        # Check if there is a reaction in the original model (model1) with the same reaction id,
        # which can be non-unique
        reac_new = reac2.copy()
        for reac in model.reactions:
            if reac2.id == reac.id:
                # Reactions in the model2 can be reversible while the same reaction in model1 is irreversible.
                # In this case, the reac2 will have cobra id XXX, equal to the reac1 cobra id XXX, but the BiGG_id will
                # be different, reac2_BiGG_id = XXXr and reac1_BiGG_id = XXX. Therefore, for those cases
                # we can add the reversible reaction to the universal database but change its cobra id to the unique
                # id XXXr, the same as the BiGG_id.
                #print(reac2.id, reac2.notes['original_bigg_ids'], reac2.reaction, reac.id, reac.notes['original_bigg_ids'], reac.reaction)
                reac_new.id = reac2.notes['original_bigg_ids'][0]
                #print(reac_new)
        model2_new_reaction_list.append(reac_new)

print('Nb of new reactions in model2: ',len(model2_new_reaction_list))
for reac in model2_new_reaction_list:
    print(reac, reac.name)

model_universal = cobra.Model("Universal")
# Add reactions to the universal model
model_universal.add_reactions(model2_new_reaction_list)

Nb of new reactions in model2:  347
3M2OBLOXRD 3-Methyl-2-oxobutanoate:lipoamide oxidoreductase(decarboxylating and acceptor-2-methylpropanoylating)
3M2OPLOXRD 3-Methyl-2-oxopentanoate:lipoamide oxidoreductase(decarboxylating and acceptor-2-methylpropanoylating)
4M2OPLOXRD 4-Methyl-2-oxopentanoate:lipoamide oxidoreductase(decarboxylating and acceptor-2-methylpropanoylating)
6PGALSZ 6-phospho-beta-galactosidase
6PHBG 6-phospho-beta-glucosidase
ABTAr 4-aminobutyrate transaminase
ACALDi Acetaldehyde dehydrogenase (acetylating)
ACGApts N-Acetyl-D-glucosamine transport via PEP:Pyr PTS
ACLDC Acetolactate decarboxylase
ACNAMt2 N-acetylneuraminate proton symport
ACOAD1 Acyl-CoA dehydrogenase (butanoyl-CoA)
ACOAD2 Acyl-CoA dehydrogenase (hexanoyl-CoA)
ACOAD3 Acyl-CoA dehydrogenase (octanoyl-CoA)
ACOAD4 Acyl-CoA dehydrogenase (decanoyl-CoA)
ACOAD5 Acyl-CoA dehydrogenase (dodecanoyl-CoA)
ACOAD6 Acyl-CoA dehydrogenase (tetradecanoyl-CoA)
ACOAD7 Acyl-CoA dehydrogenase (hexadecanoyl-CoA)
ACONT Aconi

Run the growMatch method for every possible false negative prediction of the iJO1366 model, using the universal database.

In [85]:
for index, row in knockout_df[(knockout_df.confusion == 'FN')].iterrows():
    try:
        knockout_gene = model.genes.get_by_id(row.gene_cobra_id)
        print(knockout_gene, knockout_gene.name)
        # Delete gene from the model using the cobra method cobra.manipulation.delete_model_genes.
        # Hint: set the option cumulative_deletions to False.
        cobra.manipulation.delete_model_genes(model, [knockout_gene], cumulative_deletions=False)
        # Run the growMatch method using our universal database to find gap-filling solutions
        addedReactions = cobra.flux_analysis.gapfilling.growMatch(model, model_universal, dm_rxns=False, ex_rxns=False)
        print('Solution FOUND:')
        print(addedReactions)
        print()
    except:
        pass

b0003 thrB
Solution FOUND:
[[<Reaction THRAr_reverse at 0x7f2830fc8898>]]

b0004 thrC
Solution FOUND:
[[<Reaction THRAr_reverse at 0x7f2830ca6b00>]]

b0009 mog
b0052 pdxA
b0071 leuD
b0072 leuC
b0073 leuB
b0074 leuA
b0109 nadC
b0131 panD
b0133 panC
b0134 panB
b0159 mtn
b0386 proC
Solution FOUND:
[[<Reaction EX_pro_L_LPAREN_e_RPAREN__reverse at 0x7f2830a59e48>]]

b0423 thiI
b0522 purK
b0523 purE
b0720 gltA
Solution FOUND:
[[<Reaction EX_pro_L_LPAREN_e_RPAREN__reverse at 0x7f28304f3748>]]

b0750 nadA
b0774 bioA
Solution FOUND:
[[]]

b0775 bioB
Solution FOUND:
[[]]

b0776 bioF
b0777 bioC
b0778 bioD
Solution FOUND:
[[]]

b0781 moaA
b0783 moaC
b0784 moaD
b0785 moaE
b0826 moeB
b0827 moeA
b0907 serC
b0908 aroA
b0928 aspC
b1062 pyrC
b1091 fabH
Solution FOUND:
[[]]

b1096 pabC
b1136 icd
Solution FOUND:
[[<Reaction EX_pro_L_LPAREN_e_RPAREN__reverse at 0x7f283097cb00>]]

b1260 trpA
b1261 trpB
b1262 trpC
b1263 trpD
b1264 trpE
b1281 pyrF
b1415 aldA
b1812 pabB
b2019 hisG
b2020 hisD
b2021 hisC
b2022 h

As an example, we will have a look at the suggested gap-filling solution for _argA_ false prediction.

First, delete the _argA_ from the original model. Hint: set the option cumulative_deletions to False.

In [86]:
model = cobra.io.load_json_model('iJO1366.json')
cobra.manipulation.delete_model_genes(model, [model.genes.b2818], cumulative_deletions=False)

Print the associated reactions that were knocked out.

In [87]:
for reac in model.genes.b2818.reactions:
    print(reac.name, reac.id, reac.reaction)

N-acetylglutamate synthase ACGS accoa_c + glu__L_c --> acglu_c + coa_c + h_c


Check that the original model predicts no growth in the mutant strain.

In [88]:
model.optimize()

<Solution -0.00 at 0x7f28336429e8>

Run the growMatch gap-filling method again.

In [89]:
addedReactions = cobra.flux_analysis.gapfilling.growMatch(model, model_universal, dm_rxns=False, ex_rxns=False)
addedReactions

[[<Reaction ORNTAC at 0x7f283073a5f8>]]

Print the suggested new reactions and their associated genes in S. aureus.

In [90]:
for reac in addedReactions[0]:
    print(reac.id, reac.name, reac.reaction, reac.gene_reaction_rule)
    for gene in reac.genes:
        print(gene.id, gene.name, gene.notes)

ORNTAC Ornithine transacetylase acorn_c + glu__L_c --> acglu_c + dummy_met_ORNTAC + orn_c SA_RS01060
SA_RS01060 None {'original_bigg_ids': ['SA0177']}


In prokaryotes, there are three major biosynthetic pathways for arginine [Ginesy2015]; “linear”, “recycling” and the “new” path- ways. The first four steps are common in all pathways and convert glutamate (GLU) to N-acetylornithine (Ac-ORN). In the linear pathway (Panel A below), present in E. coli and very few other species, theses reactions are catalyzed by enzymes argABCD in a linear sequence. The next step, which distinguishes the linear pathway from the other two pathways, is deacetylation of Ac-ORN by AOase (argE) to yield ORN. In many other prokaryotes ARG is synthesized via the recycling pathway (Panel B below) which is regarded as more evolved and energetically more favorable than the linear pathway. In this pathway, the first and fifth steps are linked by the recycling reaction: the acetyl group deacetylated from Ac-ORN in the fifth biosynthetic step is re-used to acetylate GLU in the first committed step. This recycling is performed by the enzyme ornithine acetyltransferase (OATase), encoded by argJ. The OATase involved in the recycling step is either monofunctional or bifunctional depending on the species.

<img src="Images/Ginesy2015_arg_pathway_12.png" width="800px" />
Adapted from [Ginesy2015]

Therefore, disruption of the glutamate biosynthesis pathway by knockout of _argA_ or _argE_ can be restored by the addition of the ornithine transacetylase reaction (ORNTAC), which performs both reactions steps in a bifunctional manner. However, there is no evidence of the existence of an ornithine acetyltransferase or _argJ_ gene in E. coli [Xu2007].

Next, check that the gap-filling method also suggests addition of ORNTAC for the _argE_ gene deletion.

First, delete _argE_ gene (`b3957`).

In [91]:
cobra.manipulation.delete_model_genes(model, [model.genes.b3957], cumulative_deletions=False)

Print the associated reactions that were knocked out.

In [92]:
for reac in model.genes.b3957.reactions:
    print(reac.name, reac.id, reac.reaction)

Acetylornithine deacetylase ACODA acorn_c + h2o_c --> ac_c + orn_c
N-acetylornithine deacetylase NACODA acg5sa_c + h2o_c --> ac_c + glu5sa_c


Check that the original model predicts no growth in the mutant strain.

In [93]:
model.optimize()

<Solution -0.00 at 0x7f2838a7acc0>

Run the growMatch gap-filling method again.

In [94]:
addedReactions = cobra.flux_analysis.gapfilling.growMatch(model, model_universal, dm_rxns=False, ex_rxns=False)
addedReactions

[[<Reaction ORNTAC at 0x7f2830f43240>]]

Print the suggested new reactions and their associated genes in S. aureus.

In [95]:
for reac in addedReactions[0]:
    print(reac.id, reac.name, reac.reaction, reac.gene_reaction_rule)
    for gene in reac.genes:
        print(gene.id, gene.name, gene.notes)

ORNTAC Ornithine transacetylase acorn_c + glu__L_c --> acglu_c + dummy_met_ORNTAC + orn_c SA_RS01060
SA_RS01060 None {'original_bigg_ids': ['SA0177']}


As a final test, we will delete both genes from the model and try to restore growth by addition of the ornithine transacetylase reaction.

Delete both _argA_ and _argE_ genes from the model. Hint: in order to apply two succesives gene deletions, set the option cumulative_deletions to True.

In [96]:
model = cobra.io.load_json_model('iJO1366.json')
cobra.manipulation.delete_model_genes(model, [model.genes.b3957], cumulative_deletions=True)
cobra.manipulation.delete_model_genes(model, [model.genes.b2818], cumulative_deletions=True)

Check that the original model predicts no growth in the mutant strain.

In [97]:
model.optimize()

<Solution 0.00 at 0x7f2830f54358>

Run the growMatch gap-filling method again.

In [98]:
addedReactions = cobra.flux_analysis.gapfilling.growMatch(model, model_universal, dm_rxns=False, ex_rxns=False)
addedReactions

[[<Reaction ORNTAC at 0x7f281fa64eb8>]]

Add the ORNTAC reaction to the model and compute growth. Hint: remove dummy metabolites from the added reactions first.

In [99]:
remove_dummy_met(addedReactions)
model.add_reactions(addedReactions[0])
model.optimize()

ORNTAC
acorn_c + glu__L_c --> acglu_c + dummy_met_ORNTAC + orn_c
acorn_c + glu__L_c --> acglu_c + orn_c



<Solution 0.98 at 0x7f28308e6828>

In order to evaluate the validity of the hypothetical new reaction, we can compare how the extended model performs with regards to the experimental knockout essentiality data. The most feasible solutions would be those that fix the most false negatives while introducing few false positives. This analysis would be a first step in assessing the feasibility of the solution, and in a second step additional experimental assays could be design to search for evidences of its existence (biochemical assays, additional knock-out assays, identification of hypothetical genes, etc).

Define the extended model with the ornithine transacetylase reaction and compute again the number of false negatives and false positives.

In [100]:
model = cobra.io.load_json_model('iJO1366.json')
model_extended = model.copy()
model_extended.add_reactions(addedReactions[0])

In [101]:
growth_rates, statuses = cobra.flux_analysis.single_gene_deletion(model_extended, model_extended.genes[:])
list(growth_rates.items())[:30]

# We convert the lists to a pandas dataframe for easier data manipulation
model_knockout_df = pd.DataFrame.from_dict({"model_extended_growth_rate": growth_rates})

# If the predicted growth rate is below the threshold, we consider the prediction as no growth.
growth_rate_threshold = 1e-5
model_knockout_df['model_extended_growth'] = model_knockout_df['model_extended_growth_rate'] > growth_rate_threshold

# Copy the gene cobra id index to a column
model_knockout_df['gene_cobra_id'] = model_knockout_df.index

print('Nb of essential genes: ',    len((model_knockout_df[~model_knockout_df['model_extended_growth']])))
print('Nb of non-essential genes: ',len((model_knockout_df[model_knockout_df['model_extended_growth']])))

model_knockout_df.drop('SA_RS01060', axis=0, inplace=True)

model_knockout_df = model_knockout_df.merge(pd.DataFrame(model_knockout_df.apply(add_gene_name, axis=1), columns=['gene']),
                                                        left_index=True, right_index=True)
model_knockout_df = model_knockout_df[ model_knockout_df.gene != 'None' ]

model_knockout_df[:10]

Nb of essential genes:  206
Nb of non-essential genes:  1162


,model_extended_growth_rate,model_extended_growth,gene_cobra_id,gene
b0002,9.835785e-01,True,b0002,thrA
b0003,6.310887e-30,False,b0003,thrB
b0004,-1.115216e-13,False,b0004,thrC
b0007,9.835785e-01,True,b0007,yaaJ
b0008,9.835785e-01,True,b0008,talB
b0009,-7.423181e-27,False,b0009,mog
b0019,9.835785e-01,True,b0019,nhaA
b0025,1.301506e-16,False,b0025,ribF
b0026,9.835785e-01,True,b0026,ileS
b0029,4.988621e-29,False,b0029,ispH


Merge results from the extended model with the previous results for the original model.

In [102]:
knockout_df2 = pd.merge(model_knockout_df, knockout_df, how='inner', on=['gene_cobra_id', 'gene_cobra_id'])
knockout_df2[:10]

,model_extended_growth_rate,model_extended_growth,gene_cobra_id,gene,model_growth_rate,model_growth,exp_growth,confusion
0,9.835785e-01,True,b0002,thrA,9.823718e-01,True,True,TP
1,6.310887e-30,False,b0003,thrB,0.000000e+00,False,True,FN
2,-1.115216e-13,False,b0004,thrC,-5.137062e-27,False,True,FN
3,9.835785e-01,True,b0007,yaaJ,9.823718e-01,True,True,TP
4,9.835785e-01,True,b0008,talB,9.823718e-01,True,True,TP
5,-7.423181e-27,False,b0009,mog,-2.027295e-29,False,True,FN
6,9.835785e-01,True,b0019,nhaA,9.823718e-01,True,True,TP
7,1.301506e-16,False,b0025,ribF,2.629892e-14,False,False,TN
8,9.835785e-01,True,b0026,ileS,9.823718e-01,True,False,FP
9,4.988621e-29,False,b0029,ispH,-2.524355e-29,False,False,TN


Compute the comparison between experimental growth and predicted growth.

In [103]:
def calculate_confusion_extended(row):
    if ( row['model_extended_growth'] and row['exp_growth'] ):
        confusion = 'TP'
    if ( row['model_extended_growth'] and not row['exp_growth'] ):
        confusion = 'FP'
    if ( not row['model_extended_growth'] and not row['exp_growth'] ):
        confusion = 'TN'
    if ( not row['model_extended_growth'] and row['exp_growth'] ):
        confusion = 'FN'
    return pd.Series({'confusion_extended':confusion})

knockout_df2 = knockout_df2.merge(knockout_df2.apply(calculate_confusion_extended, axis=1), left_index=True, right_index=True)
knockout_df2[:10]

,model_extended_growth_rate,model_extended_growth,gene_cobra_id,gene,model_growth_rate,model_growth,exp_growth,confusion,confusion_extended
0,9.835785e-01,True,b0002,thrA,9.823718e-01,True,True,TP,TP
1,6.310887e-30,False,b0003,thrB,0.000000e+00,False,True,FN,FN
2,-1.115216e-13,False,b0004,thrC,-5.137062e-27,False,True,FN,FN
3,9.835785e-01,True,b0007,yaaJ,9.823718e-01,True,True,TP,TP
4,9.835785e-01,True,b0008,talB,9.823718e-01,True,True,TP,TP
5,-7.423181e-27,False,b0009,mog,-2.027295e-29,False,True,FN,FN
6,9.835785e-01,True,b0019,nhaA,9.823718e-01,True,True,TP,TP
7,1.301506e-16,False,b0025,ribF,2.629892e-14,False,False,TN,TN
8,9.835785e-01,True,b0026,ileS,9.823718e-01,True,False,FP,FP
9,4.988621e-29,False,b0029,ispH,-2.524355e-29,False,False,TN,TN


Count number of true positive and true negatives. Hint: use the `value_counts()` method of the dataframe column.

In [104]:
confusion_table2 = knockout_df2['confusion_extended'].value_counts()
confusion_table2

TP    1070
FN     112
TN      92
FP      54
Name: confusion_extended, dtype: int64

Compare to the results of the original model.

In [105]:
confusion_table

TP    1068
FN     114
TN      92
FP      54
Name: confusion, dtype: int64

The extended model corrected 2 false negatives while maintaining the same number of false positives. However, more extended analysis should be done by analyzing the predictions with other media composition and compare the additional single gene deletions experimental data.

---------------

### Further reading

Recently, constraint-based models have been extended beyond the metabolism to account for non-metabolic processes as transcription and translation regulation [Bordbar2014]. A key challenge in the development of these new models is the integration of proteomics and transcriptomics data to parameterize the additional processes.

### References
+ Aziz, R. K., Khaw, V. L., Monk, J. M., Brunk, E., Lewis, R., Loh, S. I., … Charusanti, P. (2015). Model-driven discovery of synergistic inhibitors against E. coli and S. enterica serovar Typhimurium targeting a novel synthetic lethal pair, aldA and prpC. Frontiers in Microbiology, 6(SEP), 1–10. http://doi.org/10.3389/fmicb.2015.00958
+ Becker, S. A., & Palsson, B. Ø. (2005). Genome-scale reconstruction of the metabolic network in Staphylococcus aureus N315: an initial draft to the two-dimensional annotation. BMC Microbiology, 5(1), 8. http://doi.org/10.1186/1471-2180-5-8
+ Bordbar, A., Monk, J. M., King, Z. A., & Palsson, B. O. (2014). Constraint-based models predict metabolic and associated cellular functions. Nature Reviews Genetics, 15(2), 107–120. http://doi.org/10.1038/nrg3643
+ Burgard, a P., Nikolaev, E. V, Schilling, C. H., & Maranas, C. D. (2004). Flux Coupling Analysis of Genome-Scale Metabolic Network. Genome Research, 14, 301–312. http://doi.org/10.1101/gr.1926504.
+ Caspi, R., Altman, T., Billington, R., Dreher, K., Foerster, H., Fulcher, C. A., … Karp, P. D. (2014). The MetaCyc database of metabolic pathways and enzymes and the BioCyc collection of Pathway/Genome Databases. Nucleic Acids Research, 42(D1), D459–D471. http://doi.org/10.1093/nar/gkt1103
+ Dalebroux, Z. D., & Swanson, M. S. (2012). ppGpp: magic beyond RNA polymerase. Nature Reviews Microbiology, 10(3), 203–212. http://doi.org/10.1038/nrmicro2720
+ King, Z. A., Lu, J., Dräger, A., Miller, P., Federowicz, S., Lerman, J. A., … Lewis, N. E. (2016). BiGG Models: A platform for integrating, standardizing and sharing genome-scale models. Nucleic Acids Research, 44(D1), D515–D522. http://doi.org/10.1093/nar/gkv1049
+ Kumar, V. S., & Maranas, C. D. (2009). GrowMatch: An automated method for reconciling in silico/in vivo growth predictions. PLoS Computational Biology, 5(3), 18–20. http://doi.org/10.1371/journal.pcbi.1000308
+ Guzmán, G. I., Utrilla, J., Nurk, S., Brunk, E., Monk, J. M., Ebrahim, A., … Feist, A. M. (2015). Model-driven discovery of underground metabolic functions in Escherichia coli. Proceedings of the National Academy of Sciences of the United States of America, 112(3), 929–34. http://doi.org/10.1073/pnas.1414218112
+ Habrych, M., Rodriguez, S., & Stewart, J. D. (2002). Purification and identification of an Escherichia coli ??-keto ester reductase as 2,5-diketo-d-gluconate reductase YqhE. Biotechnology Progress, 18(2), 257–261. http://doi.org/10.1021/bp0101841
+ Latendresse, M., Krummenacker, M., Trupp, M., & Karp, P. D. (2012). Construction and completion of flux balance models from pathway databases. Bioinformatics, 28(3), 388–396. http://doi.org/10.1093/bioinformatics/btr681
+ Latendresse, M. (2014). Efficiently gap-filling reaction networks. BMC Bioinformatics, 15, 225. http://doi.org/10.1186/1471-2105-15-225
+ Mackie, A., Paley, S., Keseler, I. M., Shearer, A., Paulsen, I. T., & Karp, P. D. (2014). Addition of Escherichia coli K-12 growth observation and gene essentiality data to the ecocyc database. Journal of Bacteriology, 196(5), 982–988. http://doi.org/10.1128/JB.01209-13
+ Orth, J. D., & Palsson, B. (2012). Gap-filling analysis of the iJO1366 Escherichia coli metabolic network reconstruction for discovery of metabolic functions. BMC Systems Biology, 6, 30. http://doi.org/10.1186/1752-0509-6-30
+ Ponce-de-Léon, M., Montero, F., Peretó, J., Ponce-de-Leon, M., Montero, F., Peretó, J., … Peretó, J. (2013). Solving gap metabolites and blocked reactions in genome-scale models: application to the metabolic network of Blattabacterium cuenoti. BMC Systems Biology, 7(1), 114. http://doi.org/10.1186/1752-0509-7-114
+ Reed, J. L., Patel, T. R., Chen, K. H., Joyce, A. R., Applebee, M. K., Herring, C. D., … Palsson, B. Ø. (2006). Systems approach to refining genome annotation. Proceedings of the National Academy of Sciences of the United States of America, 103(46), 17480–17484. http://doi.org/10.1073/pnas.0603364103
+ Satish Kumar, V., Dasika, M. S., & Maranas, C. D. (2007). Optimization based automated curation of metabolic reconstructions. BMC Bioinformatics, 8, 212. http://doi.org/10.1186/1471-2105-8-212
+ Thiele, I., & Palsson, B. Ø. (2010). A protocol for generating a high-quality genome-scale metabolic reconstruction. Nature Protocols, 5(1), 93–121. http://doi.org/10.1038/nprot.2009.203
+ Yum, D. Y., Lee, B. Y., & Pan, J. G. (1999). Identification of the yqhE and yafB genes encoding two 2, 5-diketo-D-gluconate reductases in Escherichia coli. Applied and Environmental Microbiology, 65(8), 3341–6. http://doi.org/PMC91502
+ Xu, Y., Labedan, B., & Glansdorff, N. (2007). Surprising arginine biosynthesis: a reappraisal of the enzymology and evolution of the pathway in microorganisms. Microbiology and Molecular Biology Reviews : MMBR, 71(1), 36–47. http://doi.org/10.1128/MMBR.00032-06